## Préparation des données

In [1]:
import pandas as pd
import numpy as np

# Création des dataframes à partir des fichiers csv
population = pd.read_csv("fr_population.csv")
vegetaux = pd.read_csv("fr_vegetaux.csv")
animaux = pd.read_csv("fr_animaux.csv")
ssalim = pd.read_csv("fr_sousalimentation.csv")
cereales = pd.read_csv("fr_céréales.csv")

# Remplacement des valeurs nulles par "NaN"
vegetaux = vegetaux.replace(0.0,np.NaN)
animaux = animaux.replace(0.0,np.NaN)

# Insertion colonne "Type" dans "animaux" et "vegetaux" 
animaux["Type"] = "Animaux"
vegetaux["Type"] = "Végétaux"

# Assigne la valeur D pour les données vides dans la colonne Symbole
population = population.fillna(value={"Symbole": "D"})

# Création de pivots sur les dataframe "vegetaux" et "animaux" pour intégrer chaque type d'élément en colonne
vegetaux = vegetaux.pivot_table(
    index=["Code zone","Zone","Code Produit", "Produit","Type"],
    values = "Valeur", 
    columns = "Élément")

animaux = animaux.pivot_table(
    index=["Code zone","Zone","Code Produit", "Produit","Type"],
    values = "Valeur", 
    columns = "Élément")

# Concaténation des dataFrame animaux et vegetaux (même schéma de colonnes)
dispo_alim = pd.concat([animaux,vegetaux])

# Conversion en kg des colonnes du dataframe dispo_alim
for i in dispo_alim[{"Disponibilité intérieure",
                    "Nourriture",
                    "Aliments pour animaux",
                    "Pertes",
                    "Autres utilisations (non alimentaire)",
                    "Exportations - Quantité",
                    "Importations - Quantité", 
                    "Production",
                    "Variation de stock",
                    "Semences",
                    "Traitement"}] : 
    dispo_alim[i]*=  1000000 

# Suppression des régions de la Chine (code zone 96,128,41 et 214) pour éviter les redondances dans "ssalim"
ssalim = ssalim[~ssalim["Code zone"].isin([96,128,41,214])]

# Conversion de la colonne "Année" en string et remplacement de l'intervalle par l'année de début de l'intervalle
ssalim["Année"] = ssalim["Année"].astype('str') 
ssalim["Année"] = ssalim["Année"].str.slice(0,4)

# Création colonne indiquant les zones où il existe de la sous-alimentation dans "ssalim"
ssalim["Zone_ssalim"] = ssalim.Valeur.notna()

# Conversion type de donnée en float de la colonne Valeur et transformation en NaN si non convertissable ("<0.1")
ssalim['Valeur'] = pd.to_numeric(ssalim['Valeur'], errors = 'coerce')

# Conversion des données de la colonne "Valeur" dans "population"
# Unité = millions
ssalim.Valeur *= 1000000

# Conversion des données de la colonne "Valeur" dans "population" 
# Unité = 1000 personnes
population["Valeur"]*= 1000

# Changement du nom de la colonne "Valeur" en "Population" dans le dataframe population
population = population.rename(columns={'Valeur':'Population'})


## Question 1 : Calculez le nombre total d’humains sur la planète pour l'année 2013

In [2]:
# Sélection des données où la colonne Zone contient "Chine"
chine = population["Zone"].str.startswith("Chine",na = False)

# Verification Agrégat Chine en additionnant la colonne Valeur par Symbole où Zone contient "Chine" 
population[chine].groupby("Symbole").Population.sum()

Symbole
A    1416667000
D    1416667000
Name: Population, dtype: int64

#### Zone "Chine" = somme des 4 autres Zones contenant "Chine". Pour ne pas comptabiliser deux fois la Chine, on ne prendra pas la valeur pour le symbole "A"

In [3]:
# Suppression des lignes contenant les régions de la chine
population = population[~population["Code zone"].isin([96,128,41,214])]

# Additionne la colonne valeur par Symbole
population2013 = population.Population.sum()
print ("Le nombre total d'humain sur la planète en 2013 est de ",population2013,
       "soit presque", round(population2013/1000000000,0), "milliards.")


Le nombre total d'humain sur la planète en 2013 est de  6997326000 soit presque 7.0 milliards.


---
## Question 2 - Identifiez les redondances, en donnant votre réponse sous forme de formule mathématique. Illustrez cette équation avec l'exemple du blé en France.

### 1/ Disponibilité intérieure = Production + Importations - Exportations + Variation de stock = Nourriture + Aliments pour animaux + Traitement + Autres utilisations + Semences + Pertes

### 2/ Illustration avec l'exemple du blé en France

In [4]:
# Sélection du blé en France dans le dataframe vegetaux
ble_fr = vegetaux.loc[(68,'France',2511),:]

# Affiche la valeur de la disponibilité intérieure du blé en France
print("Disponibilité intérieure = ",float(ble_fr["Disponibilité intérieure"].iloc[0]))

# Calcul et affiche "Production + Importations - Exportations + Variation de stock"
print ("Production + Importations - Exportations + Variation de stock =", float(
    ble_fr["Production"].iloc[0] 
    + ble_fr["Importations - Quantité"].iloc[0] 
    + ble_fr["Variation de stock"].iloc[0] 
    - ble_fr["Exportations - Quantité"].iloc[0]))

# Calcul et affiche "Nourriture + Aliments pour animaux + Traitement + Autres utilisations + Semences + Pertes" 
print ("Nourriture + Aliments pour animaux + Traitement + Autres utilisations + Semences + Pertes =", float(
    ble_fr["Nourriture"].iloc[0] 
    + ble_fr["Aliments pour animaux"].iloc[0] 
    + ble_fr["Traitement"].iloc[0]
    + ble_fr["Autres utilisations (non alimentaire)"].iloc[0] 
    + ble_fr["Semences"].iloc[0] 
    + ble_fr["Pertes"].iloc[0]))

Disponibilité intérieure =  20298.0
Production + Importations - Exportations + Variation de stock = 20298.0
Nourriture + Aliments pour animaux + Traitement + Autres utilisations + Semences + Pertes = 20298.0


### Pour le blé en France : Disponibilité intérieure = Production + Importations - Exportations + Variation de stock = Nourriture + Aliments pour animaux + Traitement + Autres utilisations + Semences + Pertes = 20298

---
## Question 3 - Calculez (pour chaque pays et chaque produit) la disponibilité alimentaire en kcal puis en kg de protéines.

### 1/ Calcul de la disponibilité alimentaire en kcal

In [5]:
# Jointure entre dispo_alim et population
dispo_alim = pd.merge(dispo_alim,
                      population[['Code zone','Population']],
                      on = "Code zone",
                      how='inner',
                      right_index=True)

In [6]:
# Insertion colonne avec calcul de la disponibilité alimentaire annuelle en kcal
# 1 année = 365 jours
dispo_alim["Disponibilité alimentaire annuelle en kcal"] = (
    dispo_alim['Disponibilité alimentaire (Kcal/personne/jour)']
    * dispo_alim['Population']
    * 365)
dispo_alim

Aliments pour animaux  \
Code zone Zone    Code Produit Produit                Type                              
1         Arménie 2731         Viande de Bovins       Animaux                     NaN   
                  2732         Viande d'Ovins/Caprins Animaux                     NaN   
                  2733         Viande de Suides       Animaux                     NaN   
                  2734         Viande de Volailles    Animaux                     NaN   
                  2736         Abats Comestible       Animaux                     NaN   
...                                                                               ...   
351       Chine   2659         Alcool, non Comestible Végétaux                    NaN   
                  2680         Aliments pour enfants  Végétaux                    NaN   
                  2745         Miel                   Végétaux                    NaN   
                  2805         Riz (Eq Blanchi)       Végétaux           1.211700e+10   
                  2899         Miscellanees           Végétaux                    NaN   

                                                                Autres utilisations (non alimentaire)  \
Code zone Zone    Code Produit Produit                Type                                              
1         Arménie 2731         Viande de Bovins       Animaux                            2.000000e+06   
                  2732         Viande d'Ovins/Caprins Animaux                                     NaN   
                  2733         Viande de Suides       Animaux                            2.000000e+06   
                  2734         Viande de Volailles    Animaux                                     NaN   
                  2736         Abats Comestible       Animaux                            3.000000e+06   
...                                                                                               ...   
351       Chine   2659         Alcool, non Comestible Végétaux                           2.390000e+08   
                  2680         Aliments pour enfants  Végétaux                                    NaN   
                  2745         Miel                   Végétaux                                    NaN   
                  2805         Riz (Eq Blanchi)       Végétaux                           2.085000e+09   
                  2899         Miscellanees           Végétaux                                    NaN   

                                                                Disponibilité alimentaire (Kcal/personne/jour)  \
Code zone Zone    Code Produit Produit                Type                                                       
1         Arménie 2731         Viande de Bovins       Animaux                                            114.0   
                  2732         Viande d'Ovins/Caprins Animaux                                             22.0   
                  2733         Viande de Suides       Animaux                                             72.0   
                  2734         Viande de Volailles    Animaux                                             45.0   
                  2736         Abats Comestible       Animaux                                             14.0   
...                                                                                                        ...   
351       Chine   2659         Alcool, non Comestible Végétaux                                             NaN   
                  2680         Aliments pour enfants  Végétaux                                             1.0   
                  2745         Miel                   Végétaux                                             2.0   
                  2805         Riz (Eq Blanchi)       Végétaux                                           797.0   
                  2899         Miscellanees           Végétaux                                             NaN   

                                                         

### 2/ Calcul de la disponibilité alimentaire en kg de protéines

In [7]:
# Insertion colonne avec calcul de la disponibilité alimentaire annuelle en kg de proteines
# 1 année = 365 jours
# 1 g = 0,001 kg
dispo_alim["Disponibilité Alimentaire Annuelle en kg de Proteines"] = (
    dispo_alim['Disponibilité de protéines en quantité (g/personne/jour)']
    *dispo_alim['Population']
    * 365
    * 0.001)
dispo_alim

Aliments pour animaux  \
Code zone Zone    Code Produit Produit                Type                              
1         Arménie 2731         Viande de Bovins       Animaux                     NaN   
                  2732         Viande d'Ovins/Caprins Animaux                     NaN   
                  2733         Viande de Suides       Animaux                     NaN   
                  2734         Viande de Volailles    Animaux                     NaN   
                  2736         Abats Comestible       Animaux                     NaN   
...                                                                               ...   
351       Chine   2659         Alcool, non Comestible Végétaux                    NaN   
                  2680         Aliments pour enfants  Végétaux                    NaN   
                  2745         Miel                   Végétaux                    NaN   
                  2805         Riz (Eq Blanchi)       Végétaux           1.211700e+10   
                  2899         Miscellanees           Végétaux                    NaN   

                                                                Autres utilisations (non alimentaire)  \
Code zone Zone    Code Produit Produit                Type                                              
1         Arménie 2731         Viande de Bovins       Animaux                            2.000000e+06   
                  2732         Viande d'Ovins/Caprins Animaux                                     NaN   
                  2733         Viande de Suides       Animaux                            2.000000e+06   
                  2734         Viande de Volailles    Animaux                                     NaN   
                  2736         Abats Comestible       Animaux                            3.000000e+06   
...                                                                                               ...   
351       Chine   2659         Alcool, non Comestible Végétaux                           2.390000e+08   
                  2680         Aliments pour enfants  Végétaux                                    NaN   
                  2745         Miel                   Végétaux                                    NaN   
                  2805         Riz (Eq Blanchi)       Végétaux                           2.085000e+09   
                  2899         Miscellanees           Végétaux                                    NaN   

                                                                Disponibilité alimentaire (Kcal/personne/jour)  \
Code zone Zone    Code Produit Produit                Type                                                       
1         Arménie 2731         Viande de Bovins       Animaux                                            114.0   
                  2732         Viande d'Ovins/Caprins Animaux                                             22.0   
                  2733         Viande de Suides       Animaux                                             72.0   
                  2734         Viande de Volailles    Animaux                                             45.0   
                  2736         Abats Comestible       Animaux                                             14.0   
...                                                                                                        ...   
351       Chine   2659         Alcool, non Comestible Végétaux                                             NaN   
                  2680         Aliments pour enfants  Végétaux                                             1.0   
                  2745         Miel                   Végétaux                                             2.0   
                  2805         Riz (Eq Blanchi)       Végétaux                                           797.0   
                  2899         Miscellanees           Végétaux                                             NaN   

                                                         

---
## Question 4 - Calculez pour chaque produit et chaque pays, le ratio "énergie/poids" en kcal/kg à partir du poids de la disponibilité alimentaire. Vérifiez la cohérence de votre calcul en comparant ce ratio aux données disponibles sur internet (valeur calorique d'un oeuf). Calculez également le pourcentage de protéines de chaque produit pour chaque pays (ratio "poids de protéines/poids total). Vérifiez la cohérence de votre calcul en comparant ce ratio aux données disponibles sur internet (teneur en protéines de l'avoine).

### 1/ Calcul, pour chaque produit et chaque pays, du ratio "énergie/poids" en kcal/kg à partir du poids de la disponibilité alimentaire.

In [8]:
# Insertion colonne avec calcul du poids de la disponibilité alimentaire annuelle pour chaque pays
dispo_alim["Poids disponibilité alimentaire annuelle en kg"] = (
    dispo_alim['Disponibilité alimentaire en quantité (kg/personne/an)'] 
    * dispo_alim['Population'])

# Insertion colonne avec calcul du ratio "énergie/poids" (en Kcal/Kg)
dispo_alim["Ratio énergie/poids (en kcal/kg)"] = (
    dispo_alim['Disponibilité alimentaire annuelle en kcal']
    / dispo_alim['Poids disponibilité alimentaire annuelle en kg'])
dispo_alim

Aliments pour animaux  \
Code zone Zone    Code Produit Produit                Type                              
1         Arménie 2731         Viande de Bovins       Animaux                     NaN   
                  2732         Viande d'Ovins/Caprins Animaux                     NaN   
                  2733         Viande de Suides       Animaux                     NaN   
                  2734         Viande de Volailles    Animaux                     NaN   
                  2736         Abats Comestible       Animaux                     NaN   
...                                                                               ...   
351       Chine   2659         Alcool, non Comestible Végétaux                    NaN   
                  2680         Aliments pour enfants  Végétaux                    NaN   
                  2745         Miel                   Végétaux                    NaN   
                  2805         Riz (Eq Blanchi)       Végétaux           1.211700e+10   
                  2899         Miscellanees           Végétaux                    NaN   

                                                                Autres utilisations (non alimentaire)  \
Code zone Zone    Code Produit Produit                Type                                              
1         Arménie 2731         Viande de Bovins       Animaux                            2.000000e+06   
                  2732         Viande d'Ovins/Caprins Animaux                                     NaN   
                  2733         Viande de Suides       Animaux                            2.000000e+06   
                  2734         Viande de Volailles    Animaux                                     NaN   
                  2736         Abats Comestible       Animaux                            3.000000e+06   
...                                                                                               ...   
351       Chine   2659         Alcool, non Comestible Végétaux                           2.390000e+08   
                  2680         Aliments pour enfants  Végétaux                                    NaN   
                  2745         Miel                   Végétaux                                    NaN   
                  2805         Riz (Eq Blanchi)       Végétaux                           2.085000e+09   
                  2899         Miscellanees           Végétaux                                    NaN   

                                                                Disponibilité alimentaire (Kcal/personne/jour)  \
Code zone Zone    Code Produit Produit                Type                                                       
1         Arménie 2731         Viande de Bovins       Animaux                                            114.0   
                  2732         Viande d'Ovins/Caprins Animaux                                             22.0   
                  2733         Viande de Suides       Animaux                                             72.0   
                  2734         Viande de Volailles    Animaux                                             45.0   
                  2736         Abats Comestible       Animaux                                             14.0   
...                                                                                                        ...   
351       Chine   2659         Alcool, non Comestible Végétaux                                             NaN   
                  2680         Aliments pour enfants  Végétaux                                             1.0   
                  2745         Miel                   Végétaux                                             2.0   
                  2805         Riz (Eq Blanchi)       Végétaux                                           797.0   
                  2899         Miscellanees           Végétaux                                             NaN   

                                                         

### 2/ Vérification du calcul en comparant avec la valeur calorique d'un oeuf.

In [9]:
dispo_alim.reset_index(inplace=True)
mean_oeuf = dispo_alim[dispo_alim["Produit"] == "Oeufs"]["Ratio énergie/poids (en kcal/kg)"].mean()
mean_oeuf

1363.8628304280364

#### Sur Wikipedia la valeur calorique de l'oeuf est de 147 kcal pour 100g soit 1470 kcal pour 1kg, il y a un écart d'environ 106 kcal avec la valeur calorifique moyenne trouvée à partir des données (1364 kcal). 
#### L'écart peut s'expliquer avec le mode de calcul des teneurs en nutriments utilisé par la FAO. Les calculs de la FAO se basent sur des poids bruts (parties comestibles et non comestibles) et les valeurs caloriques varient en fonction des pays et de l'utilisation / transformation des produits.
#### "Il existe une grande variabilité de la composition nutritionnelle des aliments, liée à de multiples facteurs : espèces animales et cultivars végétaux, origine géographique, conditions de culture et d'élevage, nature des matières premières, recettes et formulations industrielles et ménagères, procédés et modes de fabrication, stockage et préparation des aliments, etc."
#### 

### 3/ Calcul, pour chaque produit et chaque pays, du ratio "poids de protéines/poids total" en pourcentage.

In [10]:
# Insertion colonne avec calcul du ratio "poids de protéines/poids total" (en pourcentage)
dispo_alim["Ratio poids de protéines/poids total (en %)"] = (
    dispo_alim['Disponibilité Alimentaire Annuelle en kg de Proteines']
    / dispo_alim['Poids disponibilité alimentaire annuelle en kg']
    *100)
dispo_alim

,Code zone,Zone,Code Produit,Produit,Type,Aliments pour animaux,Autres utilisations (non alimentaire),Disponibilité alimentaire (Kcal/personne/jour),Disponibilité alimentaire en quantité (kg/personne/an),Disponibilité de matière grasse en quantité (g/personne/jour),...,Production,Semences,Traitement,Variation de stock,Population,Disponibilité alimentaire annuelle en kcal,Disponibilité Alimentaire Annuelle en kg de Proteines,Poids disponibilité alimentaire annuelle en kg,Ratio énergie/poids (en kcal/kg),Ratio poids de protéines/poids total (en %)
0,1,Arménie,2731,Viande de Bovins,Animaux,NaN,2.000000e+06,114.0,19.66,8.88,...,5.400000e+07,NaN,NaN,NaN,2977000,1.238730e+11,8.660242e+06,5.852782e+07,2116.480163,14.796796
1,1,Arménie,2732,Viande d'Ovins/Caprins,Animaux,NaN,NaN,22.0,3.02,1.89,...,9.000000e+06,NaN,NaN,NaN,2977000,2.390531e+10,1.216998e+06,8.990540e+06,2658.940397,13.536424
2,1,Arménie,2733,Viande de Suides,Animaux,NaN,2.000000e+06,72.0,9.67,6.72,...,1.300000e+07,NaN,NaN,NaN,2977000,7.823556e+10,2.749111e+06,2.878759e+07,2717.683557,9.549638
3,1,Arménie,2734,Viande de Volailles,Animaux,NaN,NaN,45.0,13.35,2.82,...,8.000000e+06,NaN,NaN,NaN,2977000,4.889722e+10,4.900589e+06,3.974295e+07,1230.337079,12.330712
4,1,Arménie,2736,Abats Comestible,Animaux,NaN,3.000000e+06,14.0,4.59,0.39,...,1.500000e+07,NaN,NaN,NaN,2977000,1.521247e+10,2.444861e+06,1.366443e+07,1113.289760,17.892157
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13233,351,Chine,2659,"Alcool, non Comestible",Végétaux,NaN,2.390000e+08,NaN,NaN,NaN,...,2.000000e+08,NaN,NaN,NaN,1416667000,NaN,NaN,NaN,NaN,NaN
13234,351,Chine,2680,Aliments pour enfants,Végétaux,NaN,NaN,1.0,0.09,0.01,...,NaN,NaN,NaN,NaN,1416667000,5.170835e+11,2.068334e+07,1.275000e+08,4055.555556,16.222222
13235,351,Chine,2745,Miel,Végétaux,NaN,NaN,2.0,0.25,NaN,...,4.660000e+08,NaN,NaN,NaN,1416667000,1.034167e+12,NaN,3.541668e+08,2920.000000,NaN
13236,351,Chine,2805,Riz (Eq Blanchi),Végétaux,1.211700e+10,2.085000e+09,797.0,77.45,2.68,...,1.368730e+11,4.679000e+09,12000000.0,-3.998000e+09,1416667000,4.121155e+14,7.482198e+09,1.097209e+11,3756.036152,6.819303


### 4/ Vérification du calcul en comparant avec la teneur en protéines de l'avoine

In [11]:
mean_avoine = dispo_alim[dispo_alim["Produit"] == "Avoine"]["Ratio poids de protéines/poids total (en %)"].mean()
mean_avoine

8.764842563527464

#### Sur Wikipedia la teneur en protéines de l'avoine est de 10,7g pour 100g soit 10,7%, il y a un écart d'environ 1,9 avec la teneur protéinique moyenne trouvée à partir des données (8,76%).

#### Même constation que pour la valeur calorique des produits, l'écart peut s'expliquer avec le mode de calcul des teneurs en nutriments utilisé par la FAO, celle-ci se basant sur des poids bruts (parties comestibles et non comestibles) et la valeur nutritionnelle variant en fonction des pays et de l'utilisation / transformation des produits. 
#### "Il existe une grande variabilité de la composition nutritionnelle des aliments, liée à de multiples facteurs : espèces animales et cultivars végétaux, origine géographique, conditions de culture et d'élevage, nature des matières premières, recettes et formulations industrielles et ménagères, procédés et modes de fabrication, stockage et préparation des aliments, etc."
#### 

----
## Question 5 - Citez 5 aliments parmi les 20 aliments les plus caloriques, en utilisant le ratio énergie/poids. Citez 5 aliments parmi les 20 aliments les plus riches en protéines.

### 1/ Citez 5 aliments parmi les 20 aliments les plus caloriques, en utilisant le ratio énergie/poids.

In [12]:
produit_topkcal = dispo_alim.groupby('Produit')["Ratio énergie/poids (en kcal/kg)"].agg(['mean'])
produit_topkcal = produit_topkcal.rename(columns={"mean":"Moyenne énergie/poids"})
produit_topkcal = produit_topkcal.sort_values("Moyenne énergie/poids", ascending = False)
produit_topkcal.head(20)

,Moyenne énergie/poids
Produit,
Huiles de Foie de Poisso,11809.003531
Huile de Sésame,10192.188616
Huile de Son de Riz,9314.765053
Huile d'Olive,9301.823385
Huile de Germe de Maïs,9278.454590
Huile de Colza&Moutarde,9157.170010
Huile d'Arachide,9118.384270
Huile de Tournesol,9041.569959
Huile de Palmistes,9010.065306


#### Dans les 20 aliments les plus caloriques, il y a majoritairement des huiles: l'huile de Sésame, l'huile d'Olive, l'huile d'Arachide, l'huile de Tournesol, l'huile de Palme...

### 2/ Citez 5 aliments parmi les 20 aliments les plus riches en protéines.

In [13]:
produit_top_proteine = dispo_alim.groupby('Produit')["Ratio poids de protéines/poids total (en %)"].agg(['mean'])
produit_top_proteine = produit_top_proteine.rename(columns={"mean":"Moyenne protéines/poids"})
produit_top_proteine = produit_top_proteine.sort_values("Moyenne protéines/poids", ascending = False)
produit_top_proteine.head(20)

,Moyenne protéines/poids
Produit,
Graines Colza/Moutarde,36.500000
Soja,27.245746
Arachides Decortiquees,25.081802
Pois,22.449305
Légumineuses Autres,22.341846
Haricots,21.889927
"Plantes Oleiferes, Autre",18.788597
Sésame,18.740780
"Viande, Autre",17.813835


#### Parmis les 20 aliments les plus riches en protéines,  on trouve : le soja, les pois, les haricots, la viande de bovins et la viande de volailles...

----
## Question 6 : Calculez, pour les produits végétaux uniquement, la disponibilité intérieure mondiale exprimée en kcal.

In [14]:
# Insertion colonne calculant la disponibilité intérieur par produit et par pays en Kcal dans dispo_alim
dispo_alim["Disponibilité intérieure en Kcal"] = (
    dispo_alim["Disponibilité intérieure"]
    *dispo_alim["Ratio énergie/poids (en kcal/kg)"])

print("\n""Pour les produits végétaux uniquement, la disponibilité intérieure mondiale exprimée est de",
      dispo_alim[dispo_alim.Type == "Végétaux"]["Disponibilité intérieure en Kcal"].sum(), "kcal.")


Pour les produits végétaux uniquement, la disponibilité intérieure mondiale exprimée est de 1.216060325067501e+16 kcal.


----
## Question 7 : Combien d'humains pourraient être nourris si toute la disponibilité intérieure mondiale de produits végétaux était utilisée pour de la nourriture ? Donnez les résultats en termes de calories, puis de protéines, et exprimez ensuite ces 2 résultats en pourcentage de la population mondiale.


### 1/ Nombre d'humains qui pourraient être nourris en utilisant toute le disponibilité intérieure mondiale de produits végétaux en fonction des besoins caloriques.


### Selon les estimations de la FAO, le besoin quotidien moyen en apports caloriques d'un humain est de 2 329 kcal (pour atteindre une alimentation suffisamment énergétique).

#### Source FAO "2020 - L’ÉTAT DE LA SÉCURITÉ ALIMENTAIRE ET DE LA NUTRITION DANS LE MONDE" (p.234 à 236) https://www.fao.org/3/ca9692fr/online/ca9692fr.html#chapter-3a_1

In [15]:
# Calcul apport moyen kcal annuel
# Besoin journalier moyen kcal = 2329 kcal
# 1 année = 365 jours
apportmoyen_kcal = 2329*365

# Calcul du nombre d'humains pouvant être nourris à partir de la totalité des végétaux disponibles
nb_humain_kcal = (
    dispo_alim[dispo_alim.Type == "Végétaux"]["Disponibilité intérieure en Kcal"].sum()
    / apportmoyen_kcal)

print("\n""En fonction des besoins moyens en apports caloriques,",
      round(nb_humain_kcal/1000000000,2),
      "milliards d'humains pourraient être nourris "
      "si toute la disponibilité intérieure mondiale de végétaux était utilisée pour l'alimentation des humains. ")


En fonction des besoins moyens en apports caloriques, 14.31 milliards d'humains pourraient être nourris si toute la disponibilité intérieure mondiale de végétaux était utilisée pour l'alimentation des humains. 


### 2/ Nombre d'humains qui pourraient être nourris en utilisant toute le disponibilité intérieure mondiale de produits végétaux en fonction des besoins en protéines.

### La FAO estime l'apport moyen recommandé en protéines à 46g par personne par jour.

#### Source FAO "2020 - L’ÉTAT DE LA SÉCURITÉ ALIMENTAIRE ET DE LA NUTRITION DANS LE MONDE" (p.234 à 236) https://www.fao.org/3/ca9692fr/online/images/CA9692FR_SOFI_tabA3-1.jpg

In [16]:
# Insertion colonne calculant la disponibilité intérieur en protéines par produit et par pays dans dispo_alim
dispo_alim["Disponibilité intérieure en Protéines"] = (
    dispo_alim["Disponibilité intérieure"]
    *dispo_alim["Ratio poids de protéines/poids total (en %)"]
    /100)

print("\n""Pour les produits végétaux uniquement, la disponibilité intérieure mondiale en protéines est de ",
      round(dispo_alim[dispo_alim.Type == "Végétaux"]["Disponibilité intérieure en Protéines"].sum()/1000,2),
      "tonnes.")

# Calcul apport moyen en protéines annuel
# Besoin journalier moyen proteines = 46g
# 1 année = 365 jours
# 1g = 0,001 kg
apportmoyen_proteines = 46*365*0.001

# Calcul du nombre d'humains pouvant être nourris à partir de la totalité des végétaux disponibles
nb_humain_proteines = (
    dispo_alim[dispo_alim.Type == "Végétaux"]["Disponibilité intérieure en Protéines"].sum()
    /apportmoyen_proteines)

print("\n""En fonction des besoins en protéines,",
      round(nb_humain_proteines/1000000000,2),
      " milliards d'humains pourraient être nourris "
      "si toute la disponibilité intérieure mondiale de végétaux était utilisée pour l'alimentation des humains.")


Pour les produits végétaux uniquement, la disponibilité intérieure mondiale en protéines est de  291725530.09 tonnes.

En fonction des besoins en protéines, 17.37  milliards d'humains pourraient être nourris si toute la disponibilité intérieure mondiale de végétaux était utilisée pour l'alimentation des humains.


### 3/ Exprimez ensuite ces 2 résultats en pourcentage de la population mondiale.

In [17]:
pop2013_dispokcal = nb_humain_kcal*100/population2013
pop2013_dispoproteines = nb_humain_proteines*100/population2013

print("\n""En fonction des besoins moyens en kcal,",
      int(pop2013_dispokcal),
      "% de la population mondiale pourrait être nourrie "
      "si l'ensemble des végétaux étaient affectés à la nourriture des humains.")
print("\n""En fonction des besoins moyens en proteines,",
      int(pop2013_dispoproteines),
      "% de la population mondiale pourrait être nourrie "
      "si l'ensemble des végétaux étaient affectés à la nourriture des humains.")


En fonction des besoins moyens en kcal, 204 % de la population mondiale pourrait être nourrie si l'ensemble des végétaux étaient affectés à la nourriture des humains.

En fonction des besoins moyens en proteines, 248 % de la population mondiale pourrait être nourrie si l'ensemble des végétaux étaient affectés à la nourriture des humains.


----
## Question 8 : Combien d'humains pourraient être nourris si toute la disponibilité alimentaire en produits végétaux la nourriture végétale destinée aux animaux et les pertes de produits végétaux étaient utilisés pour de la nourriture ? Donnez les résultats en termes de calories, puis de protéines, et exprimez ensuite ces 2 résultats en pourcentage de la population mondiale.

### 1/ Nombre d'humains qui pourraient être nourris en utilisant toute le disponibilité végétale mondiale de nourriture, nourriture pour animaux et pertes en fonction des besoins caloriques.

In [18]:
# création dataframe vegetaux avec les colonnes "Nourriture", "Aliments pour animaux" et "Pertes"
vegetaux_nap = dispo_alim[dispo_alim.Type == "Végétaux"][['Code zone',
                                                          'Zone',
                                                          'Code Produit',
                                                          'Produit',
                                                          'Nourriture',
                                                          'Aliments pour animaux',
                                                          'Pertes','Ratio énergie/poids (en kcal/kg)',
                                                          'Ratio poids de protéines/poids total (en %)']]

# Insertion colonne calculant la Valeur * le Ratio énergie/poids
vegetaux_nap["Quantité NAP en Kcal"] = (
    (vegetaux_nap["Nourriture"].fillna(0) 
    + vegetaux_nap["Aliments pour animaux"].fillna(0) 
    + vegetaux_nap["Pertes"].fillna(0))
    * vegetaux_nap["Ratio énergie/poids (en kcal/kg)"])

# Calcul du nombre d'humains pouvant être nourris avec un apport suffisant en Kcal
humain_napkcal = (
    vegetaux_nap["Quantité NAP en Kcal"].sum()
    /apportmoyen_kcal)

print("\n""En fonction des besoins caloriques,",
      round(humain_napkcal/1000000000,2),
      " milliards d'humains pourraient être nourris "
      'si toute la "Nourriture, Aliments pour animaux et Pertes" mondiale de végétaux '
      "étaient utilisés pour l'alimentation des humains.")


En fonction des besoins caloriques, 10.78  milliards d'humains pourraient être nourris si toute la "Nourriture, Aliments pour animaux et Pertes" mondiale de végétaux étaient utilisés pour l'alimentation des humains.


### 2/ Nombre d'humains qui pourraient être nourris en utilisant toute le disponibilité végétale mondiale de nourriture, aliments pour animaux et pertes en fonction des besoins en protéines.

In [19]:
# Insertion colonne calculant la "Valeur" * le "Ratio poids de protéines/poids total"
vegetaux_nap["Quantité de Protéines NAP (en Kg)"] = (
    (vegetaux_nap["Nourriture"].fillna(0) 
     + vegetaux_nap["Aliments pour animaux"].fillna(0) 
     + vegetaux_nap["Pertes"].fillna(0)) 
    * vegetaux_nap["Ratio poids de protéines/poids total (en %)"]/100)


# Calcul du nombre d'humains pouvant être nourris avec un apport suffisant en protéines
humain_napproteines = (
    vegetaux_nap["Quantité de Protéines NAP (en Kg)"].sum()
    / (apportmoyen_proteines))
print("\n""En fonction des besoins en calories,",
      round(humain_napproteines/1000000000,2),
      "milliards d'humains pourraient être nourris "
      'si toute la "Nourriture, Aliments pour animaux et Pertes" mondiale de végétaux '
      "étaient utilisés pour l'alimentation des humains.")


En fonction des besoins en calories, 12.17 milliards d'humains pourraient être nourris si toute la "Nourriture, Aliments pour animaux et Pertes" mondiale de végétaux étaient utilisés pour l'alimentation des humains.


### 3/ Exprimez ensuite ces 2 résultats en pourcentage de la population mondiale.

In [20]:
pop2013_napkcal = humain_napkcal*100/population2013
pop2013_napproteines = humain_napproteines*100/population2013

print("\n""En fonction des besoins moyens en kcal,",
      int(pop2013_napkcal),
      "% de la population mondiale pourrait être nourrie "
      'si toute la "Nourriture, Aliments pour animaux et Pertes" mondiale de végétaux '
      "étaient affectés à la nourriture des humains.")
print("\n""En fonction des besoins moyens en proteines,",
      int(pop2013_napproteines),
      "% de la population mondiale pourrait être nourrie "
      'si toute la "Nourriture, Aliments pour animaux et Pertes" mondiale de végétaux '
      "étaient affectés à la nourriture des humains.")


En fonction des besoins moyens en kcal, 154 % de la population mondiale pourrait être nourrie si toute la "Nourriture, Aliments pour animaux et Pertes" mondiale de végétaux étaient affectés à la nourriture des humains.

En fonction des besoins moyens en proteines, 173 % de la population mondiale pourrait être nourrie si toute la "Nourriture, Aliments pour animaux et Pertes" mondiale de végétaux étaient affectés à la nourriture des humains.


----
## Question 9 : Combien d'humains pourraient être nourris avec la disponibilité alimentaire mondiale ? Donnez les résultats en termes de calories, puis de protéines, et exprimez ensuite ces 2 résultats en pourcentage de la population mondiale.


In [21]:
humain_kcal = (dispo_alim['Disponibilité alimentaire annuelle en kcal'].sum()
                    / apportmoyen_kcal)
humain_proteines = (dispo_alim['Disponibilité Alimentaire Annuelle en kg de Proteines'].sum()
                 / apportmoyen_proteines)

print("\n""En fonction des besoins moyens en calories,",
      round(humain_kcal/1000000000,2),
      "milliards d'humains pourraient être nourris si toute la disponibilité alimentaire était utilisée.")
print("\n""En fonction des besoins moyens en protéines,",
      round(humain_proteines/1000000000,2),
      "milliards d'humains pourraient être nourris si toute la disponibilité alimentaire était utilisée.")

# Expression en pourcentage de la la population mondiale 
pop2013_kcal = humain_kcal*100/population2013
pop2013_proteines = humain_proteines*100/population2013

print("\n""En fonction des besoins moyens en kcal,",
      int(pop2013_kcal),
      "% de la population mondiale pourrait être nourrie "
      "si toute la disponibilité alimentaire était utilisée.")
print("\n""En fonction des besoins moyens en proteines,",
      int(pop2013_proteines),
      "% de la population mondiale pourrait être nourrie si la disponibilité alimentaire était utilisée.")



En fonction des besoins moyens en calories, 8.66 milliards d'humains pourraient être nourris si toute la disponibilité alimentaire était utilisée.

En fonction des besoins moyens en protéines, 12.34 milliards d'humains pourraient être nourris si toute la disponibilité alimentaire était utilisée.

En fonction des besoins moyens en kcal, 123 % de la population mondiale pourrait être nourrie si toute la disponibilité alimentaire était utilisée.

En fonction des besoins moyens en proteines, 176 % de la population mondiale pourrait être nourrie si la disponibilité alimentaire était utilisée.


---
## Question 10 : A partir des données téléchargées qui concernent la sous-nutrition, répondez à cette question : Quelle proportion de la population mondiale est considérée comme étant en sous-nutrition ?

In [22]:
pop2013_ssalim = ssalim[ssalim["Année"]=="2013"].Valeur.sum()*100/population2013
print("\n""En 2013,",
      round(pop2013_ssalim,2),
      "% de la population mondiale était considérée en sous-nutrition, soit",
      ssalim[ssalim["Année"]=="2013"].Valeur.sum()/1000000,
      "millions de personnes.")


En 2013, 10.51 % de la population mondiale était considérée en sous-nutrition, soit 735.2 millions de personnes.


---
## Question 11 : En ne prenant en compte que les céréales destinées à l'alimentation (humaine et animale), quelle proportion (en termes de poids) est destinée à l'alimentation animale ?

In [23]:
# Création dataframe liste des céréales à partir du dataframe cereales
cereales_list = cereales[["Code Produit","Produit"]].drop_duplicates()
cereales_list.set_index("Code Produit", inplace=True)
cereales_list

,Produit
Code Produit,
2511,Blé
2805,Riz (Eq Blanchi)
2513,Orge
2514,Maïs
2517,Millet
2515,Seigle
2516,Avoine
2518,Sorgho
2520,"Céréales, Autres"


In [24]:
# Jointure entre cereales_list et dispo_alim
dispo_alim_cereales = pd.merge(dispo_alim,
                               cereales_list,
                               on=['Code Produit'], 
                               how='inner')

# Addition des céréales destinées à l'alimentation des animaux
cereales_anim = dispo_alim_cereales["Aliments pour animaux"].sum()

# Addition des céréales destinées à l'alimentation des humains
cereales_humain = dispo_alim_cereales["Poids disponibilité alimentaire annuelle en kg"].sum()

print("En 2013, la proportion de céréales destinée à l'alimentation des animaux représente",
      round(cereales_anim*100/(cereales_humain+cereales_anim),2),
      "% de la disponibilité alimentaire humaine et animale, soit",
      round(cereales_anim/1000000000,2),
      "millions de tonnes.")

En 2013, la proportion de céréales destinée à l'alimentation des animaux représente 45.91 % de la disponibilité alimentaire humaine et animale, soit 873.54 millions de tonnes.


---

## Question 12 : Donnez les 3 produits qui ont la plus grande valeur pour chacun des 2 ratios (vous aurez donc 6 produits à citer)

### 1/ Sélectionnez parmi les données des bilans alimentaires les informations relatives aux pays dans lesquels la FAO recense des personnes en sous-nutrition.

In [25]:
# Création dataframe liste des pays où il existe des personnes en sous-nutrition à partir du dataframe ssalim
pays_ssalim = ssalim[ssalim.Zone_ssalim == True][["Code zone","Zone"]].drop_duplicates()
pays_ssalim.set_index("Code zone", inplace=True)
pays_ssalim

,Zone
Code zone,
2,Afghanistan
202,Afrique du Sud
3,Albanie
4,Algérie
7,Angola
...,...
236,Venezuela (République bolivarienne du)
237,Viet Nam
249,Yémen


### 2/ Repérez les 15 produits les plus exportés par ce groupe de pays.

In [26]:
# Jointure entre pays_ssalim et dispo_alim
dispo_ssalim = pd.merge(dispo_alim,
                        pays_ssalim,
                        on=['Code zone'],
                        how='inner')

In [27]:
# Pivot avec addition sur les données d'exportations
ssalim_export = dispo_ssalim[["Code Produit","Produit","Exportations - Quantité"]].pivot_table(
    values ='Exportations - Quantité' ,
    index = ["Code Produit","Produit"], 
    aggfunc = "sum")

# création dataframe avec les 15 produits les plus exportés des les pays où il existe de la sous-nutrition
produit_export = ssalim_export.sort_values('Exportations - Quantité', ascending = False).head(15)
produit_export

,,Exportations - Quantité
Code Produit,Produit,
2514,Maïs,5.472200e+10
2577,Huile de Palme,4.636100e+10
2532,Manioc,3.593100e+10
2511,Blé,3.383100e+10
2805,Riz (Eq Blanchi),3.313300e+10
2542,Sucre Eq Brut,2.638800e+10
2605,"Légumes, Autres",2.030900e+10
2615,Bananes,1.801300e+10
2848,Lait - Excl Beurre,1.599300e+10


## 3/ Parmi les données des bilans alimentaires au niveau mondial, sélectionnez les 200 plus grandes importations de ces produits (1 importation = une quantité d'un produit donné importée par un pays donné)

In [28]:
# Jointure entre produit_export15 et dispo_alim
dispo_top15export = pd.merge(
    dispo_alim[[
        "Code zone",
        "Zone",
        "Code Produit",
        "Produit",
        "Importations - Quantité",
        "Autres utilisations (non alimentaire)",
        "Disponibilité intérieure",
        "Poids disponibilité alimentaire annuelle en kg",
        "Aliments pour animaux"]],
    produit_export, 
    on=['Code Produit'],
    how='inner')

#création dataframe avec les 200 produits
produit_import = dispo_top15export.sort_values('Importations - Quantité', ascending = False).head(200)
produit_import

,Code zone,Zone,Code Produit,Produit,Importations - Quantité,Autres utilisations (non alimentaire),Disponibilité intérieure,Poids disponibilité alimentaire annuelle en kg,Aliments pour animaux,Exportations - Quantité
2453,351,Chine,2555,Soja,6.556400e+10,1.000000e+06,7.756400e+10,5.482501e+09,9.530000e+09,1.590300e+10
2309,351,Chine,2532,Manioc,3.046600e+10,9.257000e+09,3.492500e+10,2.663334e+09,2.287300e+10,3.593100e+10
749,110,Japon,2514,Maïs,1.440300e+10,2.880000e+08,1.466100e+10,1.170996e+09,1.096400e+10,5.472200e+10
635,194,Arabie saoudite,2513,Orge,1.054700e+10,NaN,1.031700e+10,2.882900e+05,1.000000e+10,8.954000e+09
380,59,Égypte,2511,Blé,1.033100e+10,NaN,1.934100e+10,1.204828e+10,4.860000e+09,3.383100e+10
...,...,...,...,...,...,...,...,...,...,...
696,33,Canada,2514,Maïs,1.035000e+09,2.490000e+08,1.328300e+10,6.666989e+08,1.072300e+10,5.472200e+10
184,21,Brésil,2848,Lait - Excl Beurre,1.032000e+09,1.000000e+08,3.354200e+10,2.991004e+10,1.898000e+09,1.599300e+10
348,11,Autriche,2511,Blé,1.031000e+09,5.800000e+07,1.320000e+09,6.889445e+08,4.670000e+08,3.383100e+10
367,40,Chili,2511,Blé,1.022000e+09,2.000000e+06,2.097000e+09,1.845519e+09,1.450000e+08,3.383100e+10


### 4/ Groupez ces importations par produit, afin d'avoir une table contenant 1 ligne pour chacun des 15 produits. 

In [29]:
produit_import = produit_import.pivot_table(index = ["Code Produit","Produit"] ,aggfunc = "sum" )
produit_import

,,Aliments pour animaux,Autres utilisations (non alimentaire),Code zone,Disponibilité intérieure,Exportations - Quantité,Importations - Quantité,Poids disponibilité alimentaire annuelle en kg
Code Produit,Produit,,,,,,,
2511,Blé,9.871500e+10,1.512500e+10,6356,4.603270e+11,1.488564e+12,1.501400e+11,3.037221e+11
2513,Orge,2.016000e+10,1.200000e+07,1160,2.960000e+10,6.267800e+10,2.251500e+10,3.719872e+08
2514,Maïs,4.028870e+11,1.821180e+11,3965,7.011040e+11,1.477494e+12,1.024890e+11,5.595000e+10
2532,Manioc,2.515300e+10,2.035500e+10,785,6.734100e+10,1.437240e+11,3.475100e+10,1.527610e+10
2542,Sucre Eq Brut,0.000000e+00,3.009000e+09,2310,7.932100e+10,4.222080e+11,3.507100e+10,6.581438e+10
2555,Soja,1.371000e+10,5.250000e+08,2269,1.595930e+11,2.226420e+11,9.349700e+10,7.529069e+09
2577,Huile de Palme,0.000000e+00,1.974700e+10,1634,2.688600e+10,5.099710e+11,3.105500e+10,6.924490e+09
2601,Tomates,0.000000e+00,0.000000e+00,792,2.127700e+10,4.570500e+10,7.143000e+09,2.025123e+10
2605,"Légumes, Autres",3.956200e+10,1.900000e+07,1914,5.906750e+11,2.437080e+11,2.962500e+10,5.057573e+11


### 5/ Calculez pour chaque produit le ratio entre la quantité destinés aux "Autres utilisations" (Other uses) et la disponibilité intérieure.

In [30]:
produit_import['Autres utilisations / disponibilité intérieure (en %)'] = (
    produit_import["Autres utilisations (non alimentaire)"]
    *100
    /produit_import["Disponibilité intérieure"])
produit_import

,,Aliments pour animaux,Autres utilisations (non alimentaire),Code zone,Disponibilité intérieure,Exportations - Quantité,Importations - Quantité,Poids disponibilité alimentaire annuelle en kg,Autres utilisations / disponibilité intérieure (en %)
Code Produit,Produit,,,,,,,,
2511,Blé,9.871500e+10,1.512500e+10,6356,4.603270e+11,1.488564e+12,1.501400e+11,3.037221e+11,3.285708
2513,Orge,2.016000e+10,1.200000e+07,1160,2.960000e+10,6.267800e+10,2.251500e+10,3.719872e+08,0.040541
2514,Maïs,4.028870e+11,1.821180e+11,3965,7.011040e+11,1.477494e+12,1.024890e+11,5.595000e+10,25.975889
2532,Manioc,2.515300e+10,2.035500e+10,785,6.734100e+10,1.437240e+11,3.475100e+10,1.527610e+10,30.226756
2542,Sucre Eq Brut,0.000000e+00,3.009000e+09,2310,7.932100e+10,4.222080e+11,3.507100e+10,6.581438e+10,3.793447
2555,Soja,1.371000e+10,5.250000e+08,2269,1.595930e+11,2.226420e+11,9.349700e+10,7.529069e+09,0.328962
2577,Huile de Palme,0.000000e+00,1.974700e+10,1634,2.688600e+10,5.099710e+11,3.105500e+10,6.924490e+09,73.447147
2601,Tomates,0.000000e+00,0.000000e+00,792,2.127700e+10,4.570500e+10,7.143000e+09,2.025123e+10,0.000000
2605,"Légumes, Autres",3.956200e+10,1.900000e+07,1914,5.906750e+11,2.437080e+11,2.962500e+10,5.057573e+11,0.003217


### 6/ Calculez pour chaque produit le ratio entre la quantité destinée à la nourriture animale et la quantité destinée à la nourriture (animale + humaine)

In [31]:
produit_import['Quantité nourriture animale / quantité nourriture (animale + humaine) (en %)'] = (
    produit_import["Aliments pour animaux"]
    *100
    /(produit_import["Aliments pour animaux"]
      +produit_import["Poids disponibilité alimentaire annuelle en kg"]))
produit_import

,,Aliments pour animaux,Autres utilisations (non alimentaire),Code zone,Disponibilité intérieure,Exportations - Quantité,Importations - Quantité,Poids disponibilité alimentaire annuelle en kg,Autres utilisations / disponibilité intérieure (en %),Quantité nourriture animale / quantité nourriture (animale + humaine) (en %)
Code Produit,Produit,,,,,,,,,
2511,Blé,9.871500e+10,1.512500e+10,6356,4.603270e+11,1.488564e+12,1.501400e+11,3.037221e+11,3.285708,24.529298
2513,Orge,2.016000e+10,1.200000e+07,1160,2.960000e+10,6.267800e+10,2.251500e+10,3.719872e+08,0.040541,98.188255
2514,Maïs,4.028870e+11,1.821180e+11,3965,7.011040e+11,1.477494e+12,1.024890e+11,5.595000e+10,25.975889,87.806127
2532,Manioc,2.515300e+10,2.035500e+10,785,6.734100e+10,1.437240e+11,3.475100e+10,1.527610e+10,30.226756,62.215085
2542,Sucre Eq Brut,0.000000e+00,3.009000e+09,2310,7.932100e+10,4.222080e+11,3.507100e+10,6.581438e+10,3.793447,0.000000
2555,Soja,1.371000e+10,5.250000e+08,2269,1.595930e+11,2.226420e+11,9.349700e+10,7.529069e+09,0.328962,64.550853
2577,Huile de Palme,0.000000e+00,1.974700e+10,1634,2.688600e+10,5.099710e+11,3.105500e+10,6.924490e+09,73.447147,0.000000
2601,Tomates,0.000000e+00,0.000000e+00,792,2.127700e+10,4.570500e+10,7.143000e+09,2.025123e+10,0.000000,0.000000
2605,"Légumes, Autres",3.956200e+10,1.900000e+07,1914,5.906750e+11,2.437080e+11,2.962500e+10,5.057573e+11,0.003217,7.254833


### 7/ Donnez les 3 produits qui ont la plus grande valeur pour chacun des 2 ratios (vous aurez donc 6 produits à citer)

In [32]:
# affichage des 3 produits qui ont la plus grande valeur pour le ratio "Autres utilisations/disponibilité intérieure"
produit_import.reset_index(inplace = True)
produit_import[["Code Produit","Produit",'Autres utilisations / disponibilité intérieure (en %)']].sort_values(
    'Autres utilisations / disponibilité intérieure (en %)',
    ascending = False).head(3)

,Code Produit,Produit,Autres utilisations / disponibilité intérieure (en %)
6,2577,Huile de Palme,73.447147
3,2532,Manioc,30.226756
2,2514,Maïs,25.975889


In [33]:
# affichage des 3 produits qui ont la plus grande valeur pour le ratio "Quantité destinée à la nourriture animale/Quantité destinée à la nourriture (animale + humaine) (en %)"
produit_import[["Code Produit",
                "Produit",
                "Quantité nourriture animale / quantité nourriture (animale + humaine) (en %)"
               ]].sort_values(
    'Quantité nourriture animale / quantité nourriture (animale + humaine) (en %)',
    ascending = False).head(3)

,Code Produit,Produit,Quantité nourriture animale / quantité nourriture (animale + humaine) (en %)
1,2513,Orge,98.188255
2,2514,Maïs,87.806127
12,2763,Poissons Pelagiques,76.346906


---
## Question 13 : Combien de tonnes de céréales pourraient être libérées si les USA diminuaient leur production de produits animaux de 10% ?

In [34]:
# Addition des céréales utilisées pour l'alimentation des animaux aux USA
alim_animaux_usa = dispo_alim_cereales[dispo_alim_cereales["Code zone"] == 231]["Aliments pour animaux"].sum()/1000

print(alim_animaux_usa*0.1,
      "tonnes de céréales pourraient être libérées "
      "si les USA diminuaient leur production de produits animaux de 10%.") 

14009600.0 tonnes de céréales pourraient être libérées si les USA diminuaient leur production de produits animaux de 10%.


---
## Question 14 : En Thaïlande, quelle proportion de manioc est exportée ? Quelle est la proportion de personnes en sous-nutrition?

In [35]:
# Calcul de la proportion de l'exportation du Manioc sur production+importation+variation de stock
# Manioc Code Produit = 2532
# Thaïlande Code zone = 216
manioc_export = float(
    dispo_alim[(dispo_alim["Code zone"]== 216)
               &(dispo_alim["Code Produit"]==2532)]["Exportations - Quantité"]
    *100
    /(dispo_alim[(dispo_alim["Code zone"]== 216)
                 &(dispo_alim["Code Produit"]==2532)]["Production"]
      + dispo_alim[(dispo_alim["Code zone"]== 216)
                   &(dispo_alim["Code Produit"]==2532)]["Importations - Quantité"]
      + dispo_alim[(dispo_alim["Code zone"]== 216)
                   &(dispo_alim["Code Produit"]==2532)]["Variation de stock"].fillna(0)
     ))

# Population en sous-nutrition en Thaïlande (Code zone = 216) en 2013
pop2013_thailande_ssalim = int(ssalim.Valeur[
    (ssalim["Code zone"]==216)
    &(ssalim["Année"]=='2013')])

# Population en Thaïlande en 2013
pop2013_thailande = int(population.Population[population["Code zone"]==216])

print("\n""En Thaïlande,", 
      round(manioc_export,2), "% de Manioc est exporté, alors que", 
      round(pop2013_thailande_ssalim*100/pop2013_thailande,2),
      "% de la population est en sous-nutrition, soit", 
      pop2013_thailande_ssalim/1000000, "millions de personnes.")


En Thaïlande, 80.1 % de Manioc est exporté, alors que 8.21 % de la population est en sous-nutrition, soit 5.5 millions de personnes.


In [37]:
# Export en fichiers csv pour la suite des questions
population.to_csv('population.csv',index = False)
dispo_alim.to_csv('dispo_alim.csv', index = False)
ssalim.to_csv('ssalim.csv', index = False)